# Introduction to Pandas, Part 2

Advanced Pandas syntax, aggregation, and joining.

In [10]:
import numpy as np
import pandas as pd

Let's start by loading the California baby names again.

In [11]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(5)

,State,Sex,Year,Name,Count
3637,CA,F,1919,Opal,17
332055,CA,M,1997,Roderick,27
43172,CA,F,1960,Marleen,11
43244,CA,F,1960,Shana,10
102204,CA,F,1986,Thalia,6


In [12]:
!ls

babynamesbystate.zip  sample_data


In [13]:
(
    babynames.query('Sex == "M" and Year == 2020')
.sort_values("Count", ascending = False)
)

,State,Sex,Year,Name,Count
395116,CA,M,2020,Noah,2625
395117,CA,M,2020,Liam,2420
395118,CA,M,2020,Mateo,2073
395119,CA,M,2020,Sebastian,1994
395120,CA,M,2020,Julian,1687
...,...,...,...,...,...
397667,CA,M,2020,Gannon,5
397666,CA,M,2020,Galen,5
397665,CA,M,2020,Fitzgerald,5
397664,CA,M,2020,Fermin,5


In [14]:
babynames.query('Sex == "M" and Year == 2020').sort_values("Count", ascending = False)


,State,Sex,Year,Name,Count
395116,CA,M,2020,Noah,2625
395117,CA,M,2020,Liam,2420
395118,CA,M,2020,Mateo,2073
395119,CA,M,2020,Sebastian,1994
395120,CA,M,2020,Julian,1687
...,...,...,...,...,...
397667,CA,M,2020,Gannon,5
397666,CA,M,2020,Galen,5
397665,CA,M,2020,Fitzgerald,5
397664,CA,M,2020,Fermin,5


In [15]:
#Guess we need the parathesis for chaining commands together???
(babynames.query('Sex == "M" and Year == 2020')
.sort_values("Count", ascending = False)
)

,State,Sex,Year,Name,Count
395116,CA,M,2020,Noah,2625
395117,CA,M,2020,Liam,2420
395118,CA,M,2020,Mateo,2073
395119,CA,M,2020,Sebastian,1994
395120,CA,M,2020,Julian,1687
...,...,...,...,...,...
397667,CA,M,2020,Gannon,5
397666,CA,M,2020,Galen,5
397665,CA,M,2020,Fitzgerald,5
397664,CA,M,2020,Fermin,5


In [16]:
(babynames.query('Sex == "M" and Year == 2020').sort_values("Name", ascending = False)
)

,State,Sex,Year,Name,Count
396944,CA,M,2020,Zyon,9
397908,CA,M,2020,Zymir,5
397074,CA,M,2020,Zyan,8
395829,CA,M,2020,Zyaire,37
396553,CA,M,2020,Zyair,13
...,...,...,...,...,...
396824,CA,M,2020,Aamir,9
397550,CA,M,2020,Aalam,5
397076,CA,M,2020,Aaditya,7
397075,CA,M,2020,Aadi,7


In [17]:
## using len(x) doesn't work here, perhaps because it's treating x as the whole series and returning the len of the whole series?
# (
# babynames.query('Sex == "M" and Year == 2020')
#          .sort_values("Name", key = lambda x: len(x),#str.len(),
#                       ascending = False)
# )

---

In [18]:
# using a Series method (.str.len()) works
(
babynames.query('Sex == "M" and Year == 2020')
         .sort_values("Name", key = lambda x: x.str.len(),
                      ascending = False)
)

,State,Sex,Year,Name,Count
397199,CA,M,2020,Michaelangelo,7
396683,CA,M,2020,Michelangelo,11
396682,CA,M,2020,Maximilliano,11
396623,CA,M,2020,Abdulrahman,11
395166,CA,M,2020,Christopher,758
...,...,...,...,...,...
397745,CA,M,2020,Ky,5
397121,CA,M,2020,Cj,7
396001,CA,M,2020,Ty,25
397401,CA,M,2020,Jj,6


An alternate approach is to create a temporary column corresponding to the length

In [19]:
#create a new series of only the lengths
babyname_lengths = babynames["Name"].str.len()

#add that series to the dataframe as a column
babynames["name_lengths"] = babyname_lengths
babynames.head(5)

,State,Sex,Year,Name,Count,name_lengths
0,CA,F,1910,Mary,295,4
1,CA,F,1910,Helen,239,5
2,CA,F,1910,Dorothy,220,7
3,CA,F,1910,Margaret,163,8
4,CA,F,1910,Frances,134,7


In [20]:
## or do the above with one line

babynames["nl"] = babynames["Name"].str.len()
babynames.head()

,State,Sex,Year,Name,Count,name_lengths,nl
0,CA,F,1910,Mary,295,4,4
1,CA,F,1910,Helen,239,5,5
2,CA,F,1910,Dorothy,220,7,7
3,CA,F,1910,Margaret,163,8,8
4,CA,F,1910,Frances,134,7,7


In [21]:
#sort by the temporary column
babynames = babynames.sort_values(by = "name_lengths", ascending=False)
babynames.head(25)

,State,Sex,Year,Name,Count,name_lengths,nl
313143,CA,M,1989,Franciscojavier,6,15,15
333732,CA,M,1997,Ryanchristopher,5,15,15
330421,CA,M,1996,Franciscojavier,8,15,15
323615,CA,M,1993,Johnchristopher,5,15,15
310235,CA,M,1988,Franciscojavier,10,15,15
323728,CA,M,1993,Ryanchristopher,5,15,15
308802,CA,M,1987,Franciscojavier,5,15,15
340695,CA,M,2000,Franciscojavier,6,15,15
102498,CA,F,1986,Mariadelosangel,5,15,15
318170,CA,M,1991,Franciscojavier,6,15,15


In [22]:
#with just 2020 boy names

babynames.query('Sex == "M" and Year == 2020').sort_values(by = "nl", ascending=False)

,State,Sex,Year,Name,Count,name_lengths,nl
397199,CA,M,2020,Michaelangelo,7,13,13
396683,CA,M,2020,Michelangelo,11,12,12
396682,CA,M,2020,Maximilliano,11,12,12
397776,CA,M,2020,Maxemiliano,5,11,11
396106,CA,M,2020,Miguelangel,21,11,11
...,...,...,...,...,...,...,...
397121,CA,M,2020,Cj,7,2,2
396828,CA,M,2020,Aj,9,2,2
397282,CA,M,2020,An,6,2,2
396465,CA,M,2020,Om,14,2,2


In [23]:
#drop the temporary column
babynames = babynames.drop("name_lengths", axis = 'columns')  # must specify "columns" (or 1) here b/c defaults to "row" (or 0)
babynames.head(5)

,State,Sex,Year,Name,Count,nl
313143,CA,M,1989,Franciscojavier,6,15
333732,CA,M,1997,Ryanchristopher,5,15
330421,CA,M,1996,Franciscojavier,8,15
323615,CA,M,1993,Johnchristopher,5,15
310235,CA,M,1988,Franciscojavier,10,15


We can also use the Python map function if we want to use an arbitrarily defined function. Suppose we want to sort by the number of occurrences of "dr" plus the number of occurences of "ea".

In [24]:
def dr_ea_count(string):
    return string.count('dr') + string.count('ea')

#create the temporary column
babynames["dr_ea_count"] = babynames["Name"].map(dr_ea_count)

#sort by the temporary column
babynames = babynames.sort_values(by = "dr_ea_count", ascending=False)
babynames.head()

,State,Sex,Year,Name,Count,nl,dr_ea_count
101969,CA,F,1986,Deandrea,6,8,3
304390,CA,M,1985,Deandrea,6,8,3
131022,CA,F,1994,Leandrea,5,8,3
115950,CA,F,1990,Deandrea,5,8,3
108723,CA,F,1988,Deandrea,5,8,3


In [25]:
def dr_ea_count(string):
    return string.count('dr') + string.count('ea')

babynames["dr_ea_count"] = babynames["Name"].map(dr_ea_count)
babynames = babynames.sort_values(by = "dr_ea_count", ascending=False)

In [26]:
#drop that column
babynames = babynames.drop("dr_ea_count", axis = 'columns')
babynames.head(5)

,State,Sex,Year,Name,Count,nl
101969,CA,F,1986,Deandrea,6,8
131022,CA,F,1994,Leandrea,5,8
115950,CA,F,1990,Deandrea,5,8
108723,CA,F,1988,Deandrea,5,8
304390,CA,M,1985,Deandrea,6,8


---

###Female Name whose popularity has dropped the most.

In this exercise, let's find the female name whose popularity has dropped the most since its peak. As an example of a name that has fallen into disfavor, consider "Jennifer", visualized below.

Note: We won't cover plotly in lecture until week 4 after Lisa covers EDA and Regex.

In [27]:
import plotly.express as px

#must resort the dataframe by Year
babynames = babynames.sort_values(by = "Year")

fig = px.line(babynames.query("Name == 'Jennifer' and Sex == 'F'"), x = "Year", y = "Count")
fig.update_layout(font_size = 18)

To answer this question, we'll need a mathematical definition for the change in popularity of a name.

For the purposes of lecture, let’s use the RTP or ratio_to_peak. This is the current count of the name divded by its maximum ever count.

Getting the max Jennifer is easy enough.

In [28]:
max_jennifers = max(babynames.query("Name == 'Jennifer' and Sex == 'F'")["Count"])
max_jennifers

6065

And we can get the most recent Jennifer count with `iloc[-1]` (TODO: Maybe make this an exercise)

In [29]:
current_jennifers = babynames.query("Name == 'Jennifer' and Sex == 'F'")["Count"].iloc[-1]
current_jennifers

91

In [30]:
#most recent Year is 2021 -- it's 2020 in the lecture

babynames.tail()

,State,Sex,Year,Name,Count,nl
234616,CA,F,2021,Jazlene,8,7
233820,CA,F,2021,Milania,14,7
233823,CA,F,2021,Monique,14,7
234869,CA,F,2021,Lovelyn,7,7
232463,CA,F,2021,Capri,108,5


In [31]:
#using 2020 to follow the lecture

current_jennifers = babynames.query("Name == 'Jennifer' and Sex == 'F' and Year == 2020")["Count"]
current_jennifers

##returns a Series object, but need an integer...he uses iloc, but there's got to be another way!

babynames.query("Name == 'Jennifer' and Sex == 'F'")
current_jennifers = babynames.query("Name == 'Jennifer' and Sex == 'F'")["Count"].iloc[-2]
current_jennifers

141

In [32]:
##another option for pulling out an integer (or one value) vice a Series

babynames.query("Name == 'Jennifer' and Sex == 'F' and Year == 2020")["Count"]  #this gives a Series of just the one number
babynames.query("Name == 'Jennifer' and Sex == 'F' and Year == 2020")["Count"].values[0]  #this pulls out the number

141

In [33]:
current_jennifers / max_jennifers

0.023248145094806265

We can also write a function that produces the ratio_to_peak for a given series.

Here for clarity, let's regenerate the `jennifer_counts` Series, but let's do so on a DataFrame where the index is the year.

In [34]:
# I don't like this function -- maybe use max year to filter??

def ratio_to_peak(series):
    return series.iloc[-1] / max(series)

In [35]:
jennifer_counts_series = babynames.query("Name == 'Jennifer' and Sex == 'F'")["Count"]
ratio_to_peak(jennifer_counts_series)

0.015004122011541632

We can try out various names below: 

In [36]:
ratio_to_peak(babynames.query("Name == 'Jessica' and Sex == 'F'")["Count"])

0.01524741081703107

Since we're only working with female names, let's create a DataFrame with only female names to simplify our later code.

In [37]:
female_babynames = babynames[babynames["Sex"] == "F"] #could have also used query but I want to use various types of syntax to increase your exposure to other ways of writing code

### Approach 1: Naive For Loop

As a first approach, we can try to use a for loop.

In [38]:
#build dictionary where each entry is the rtp for a given name
#e.g. rtps["jennifer"] should be 0.0231
#rtps = {}
#for name in ??:
#    counts_of_current_name = female_babynames[??]["Count"]
#    rtps[name] = ratio_to_peak(counts_of_current_name)
    
#convert to series
#rtps = pd.Series(rtps) 
#rtps

Answer below. Note that we only used the first 100 names because otherwise the code takes ages to complete running.

In [39]:
df = pd.DataFrame(
    {
        "A": [1, 1, 2, 2],
        "B": [1, 2, 3, 4],
        "C": [0.362838, 0.227877, 1.267767, -0.562860],
    }
)
df.groupby('A').agg('min')

,B,C
A,,
1,1,0.227877
2,3,-0.562860


In [40]:
female_babynames

,State,Sex,Year,Name,Count,nl
46,CA,F,1910,Jessie,32,6
171,CA,F,1910,Daisy,7,5
8,CA,F,1910,Virginia,101,8
80,CA,F,1910,Vivian,22,6
63,CA,F,1910,Charlotte,26,9
...,...,...,...,...,...,...
234616,CA,F,2021,Jazlene,8,7
233820,CA,F,2021,Milania,14,7
233823,CA,F,2021,Monique,14,7
234869,CA,F,2021,Lovelyn,7,7


In [41]:
#build dictionary where entry i is the ammd for the given name
#e.g. rtps["jennifer"] should be 0.0231
rtps = {}
for name in female_babynames["Name"].unique()[0:50]:
    counts_of_current_name = female_babynames[female_babynames["Name"] == name]["Count"]
    rtps[name] = ratio_to_peak(counts_of_current_name)
    
#convert to series
rtps = pd.Series(rtps) 
rtps

Jessie        0.346939
Daisy         0.271893
Virginia      0.044586
Vivian        0.476309
Charlotte     0.814975
Agnes         0.204301
Edna          0.031447
Sadie         0.549261
Florence      0.124161
Emily         0.320550
Helene        0.138889
Patricia      0.009306
Katherine     0.166434
Merle         0.208333
Angie         0.191667
Celia         0.509804
Marguerite    0.090909
Dora          0.131387
Kathryn       0.048452
Gloria        0.074777
Amy           0.112259
Jean          0.032609
Jacqueline    0.070468
Cecile        0.291667
Viola         0.226804
Sally         0.040000
Alyce         0.272727
Minnie        0.121951
Laura         0.031910
Vera          0.759690
Mabel         0.807229
Rosemary      0.309963
Jeannette     0.037313
Esther        0.509294
Margarita     0.089494
Janet         0.010283
Willie        0.260870
Amelia        1.000000
Aileen        0.457031
Ethel         0.038760
Elinor        0.387755
Rosalie       0.747826
Joyce         0.059659
Adeline    

In [42]:
rtps.sort_values()

Patricia      0.009306
Janet         0.010283
Edna          0.031447
Laura         0.031910
Jean          0.032609
Jeannette     0.037313
Ethel         0.038760
Sally         0.040000
Virginia      0.044586
Kathryn       0.048452
Joyce         0.059659
Jacqueline    0.070468
Katie         0.073203
Gloria        0.074777
Verna         0.086207
Margarita     0.089494
Marguerite    0.090909
Amy           0.112259
Minnie        0.121951
Florence      0.124161
Dora          0.131387
Helene        0.138889
Elaine        0.152125
Katherine     0.166434
Angie         0.191667
Agnes         0.204301
Merle         0.208333
Viola         0.226804
Della         0.230769
Willie        0.260870
Daisy         0.271893
Alyce         0.272727
Althea        0.285714
Cecile        0.291667
Rosemary      0.309963
Emily         0.320550
Jessie        0.346939
Elinor        0.387755
Aileen        0.457031
Vivian        0.476309
Esther        0.509294
Celia         0.509804
Sadie         0.549261
Adeline    

### Approach 2: Use groupby.agg

Instead, we can use the very powerful groupby.agg operation, which allows us to simply and efficiently compute what we want.

In [43]:
female_babynames.head()

,State,Sex,Year,Name,Count,nl
46,CA,F,1910,Jessie,32,6
171,CA,F,1910,Daisy,7,5
8,CA,F,1910,Virginia,101,8
80,CA,F,1910,Vivian,22,6
63,CA,F,1910,Charlotte,26,9


In [44]:
female_babynames

,State,Sex,Year,Name,Count,nl
46,CA,F,1910,Jessie,32,6
171,CA,F,1910,Daisy,7,5
8,CA,F,1910,Virginia,101,8
80,CA,F,1910,Vivian,22,6
63,CA,F,1910,Charlotte,26,9
...,...,...,...,...,...,...
234616,CA,F,2021,Jazlene,8,7
233820,CA,F,2021,Milania,14,7
233823,CA,F,2021,Monique,14,7
234869,CA,F,2021,Lovelyn,7,7


In [45]:
rtp_table = female_babynames.groupby("Name").agg('count')
female_babynames.set_index('Name').loc['Aadhira', :]['Year']

female_babynames.groupby("Name").agg('count')

#Note: If this cell crashes, comment out the code and use the female_babynames.groupby("Name")[["Count"]].agg(ratio_to_peak) instead

,State,Sex,Year,Count,nl
Name,,,,,
Aadhira,4,4,4,4,4
Aadhya,13,13,13,13,13
Aadya,14,14,14,14,14
Aahana,13,13,13,13,13
Aahna,1,1,1,1,1
...,...,...,...,...,...
Zyanya,22,22,22,22,22
Zyla,12,12,12,12,12
Zylah,11,11,11,11,11


This is simply the equivalent of [http://data8.org/datascience/_autosummary/datascience.tables.Table.group.html](http://data8.org/datascience/_autosummary/datascience.tables.Table.group.html) from Data8, e.g. if babynames were using `Table`, our code would read:

`female_babynames.group("Name", ratio_to_peak)`

For a visual review of groupby, see this [lecture slide](https://docs.google.com/presentation/d/1FC-cs5MTGSkDzI_7R_ZENgwoHQ4aVamxFOpJuWT0fo0/edit#slide=id.g477ed0f02e_0_390).

If you're using an early enough version of pandas, the code above will not crash, and will automatically drop columns for which `ratio_to_peak` fails, e.g. the Sex column. 

However, according to a warning message that is generated as of January 2022, at some point this code will no longer be considered valid Pandas code, and the code will crash on columns for which the aggregation function is undefined. 

Whether we're trying to avoid a crash, or just want a clean DataFrame, let's explicitly select only the Count column. The idea here is that we don't really care about the meaningless Year column, so we may as well exclude it when we compute our `ratio_to_peak` values. 

In [46]:
rtp_table = female_babynames.groupby("Name")[["Count"]].agg(ratio_to_peak)
rtp_table

,Count
Name,
Aadhira,0.700000
Aadhya,0.580000
Aadya,0.724138
Aahana,0.192308
Aahna,1.000000
...,...
Zyanya,0.857143
Zyla,1.000000
Zylah,1.000000


In [47]:
# this code renames the Count column to RTP. You'll see this syntax in lab
rtp_table = rtp_table.rename(columns = {"Count": "Count RTP"})
rtp_table

,Count RTP
Name,
Aadhira,0.700000
Aadhya,0.580000
Aadya,0.724138
Aahana,0.192308
Aahna,1.000000
...,...
Zyanya,0.857143
Zyla,1.000000
Zylah,1.000000


In [48]:
rtp_table.sort_values("Count RTP")

,Count RTP
Name,
Debra,0.001260
Susan,0.002034
Debbie,0.002817
Cheryl,0.003273
Carol,0.003635
...,...
Jovi,1.000000
Neta,1.000000
Doni,1.000000


In [49]:
fig = px.line(babynames.query("Name == 'Debra' and Sex == 'F'"), x = "Year", y = "Count")
fig.update_layout(font_size = 16)

In [50]:
top10 = rtp_table.sort_values("Count RTP").head(10).index
top10

Index(['Debra', 'Susan', 'Debbie', 'Cheryl', 'Carol', 'Tammy', 'Terri',
       'Shannon', 'Deborah', 'Carolyn'],
      dtype='object', name='Name')

In [51]:
fig = px.line(babynames.query("Name in @top10 and Sex == 'F'"), x = "Year", y = "Count", color = "Name")
fig.update_layout(font_size = 13)

---

## Some Additional Groupby Puzzles

Groupby puzzle #1: Try to create a groupby.agg call that gives the total babies born with each name.

In [52]:
puzzle1 = female_babynames.groupby("Name")[["Count"]].agg(sum)
puzzle1

,Count
Name,
Aadhira,29
Aadhya,397
Aadya,251
Aahana,134
Aahna,7
...,...
Zyanya,186
Zyla,124
Zylah,109


Groupby puzzle #2: Try to create a groupby.agg call that gives total babies born in each year.

In [53]:
puzzle2 = female_babynames.groupby("Year")[["Count"]].agg(sum)  # all female babynames
puzzle2 = babynames.groupby("Year")[["Count"]].agg(sum)  #both male and femaile babynames

puzzle2

,Count
Year,
1910,9163
1911,9983
1912,17946
1913,22094
1914,26926
...,...
2017,410835
2018,395151
2019,386504


We can write this using a groupby shorthand aggregation method. Here, `sum()` is shorthand for `groupby.agg(sum)`.

In [54]:
puzzle2 = babynames.groupby("Year")[["Count"]].sum()

In [55]:
fig = px.line(puzzle2, y = "Count")
fig.update_layout(font_size = 15)

In [56]:
#not covered in lecture but we can also compute relative births
relative_births = puzzle2 / max(puzzle2["Count"])

In [57]:
fig = px.line(relative_births, y = "Count")
fig.update_layout(font_size = 15)

#### groupby Puzzle #4

In [58]:
elections = pd.read_csv("https://raw.githubusercontent.com/archenclan/dsKal100/main/lec/lec04/elections.csv")
elections.sample(5)

,Year,Candidate,Party,Popular vote,Result,%
123,1976,Jimmy Carter,Democratic,40831881,win,50.271900
80,1924,Calvin Coolidge,Republican,15723789,win,54.329113
152,2000,George W. Bush,Republican,50456002,win,47.974666
69,1912,William Taft,Republican,3486242,loss,23.218466
124,1976,Lester Maddox,American Independent,170274,loss,0.209640


In [59]:
elections.query("Candidate == 'Woodrow Wilson'")

,Year,Candidate,Party,Popular vote,Result,%
70,1912,Woodrow Wilson,Democratic,6296284,win,41.933422
74,1916,Woodrow Wilson,Democratic,9126868,win,49.367987


We have to be careful when using aggregation functions. For example, the code below might be misinterpreted to say that Woodrow Wilson successfully ran for election in 2020. Why is this happening?

In [60]:
elections.groupby("Party").agg(max).head(10)

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1976,Thomas J. Anderson,873053,loss,21.554001
American Independent,1976,Lester Maddox,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182
Communist,1932,William Z. Foster,103307,loss,0.261069
Constitution,2016,Michael Peroutka,203091,loss,0.152398
Constitutional Union,1860,John Bell,590901,loss,12.639283
Democratic,2020,Woodrow Wilson,81268924,win,61.344703


### Quick Subpuzzle

Inspired by above, try to predict the results of the groupby operation shown. The answer is below the image.

![groupby_puzzle.png](attachment:groupby_puzzle.png)

The top ?? will be "hi", the second ?? will be "tx", and the third ?? will be "sd". 

In [61]:
ds = pd.DataFrame(dict(x=[3,1,4,1,5,9,2,5,6], 
                      y=['ak', 'tx', 'fl', 'hi', 'mi', 'ak', 'ca', 'sd', 'nc']), 
                      index=list('ABCABCACB') )
ds

,x,y
A,3,ak
B,1,tx
C,4,fl
A,1,hi
B,5,mi
C,9,ak
A,2,ca
C,5,sd
B,6,nc


In [62]:
ds.groupby(ds.index).agg(max)

,x,y
A,3,hi
B,6,tx
C,9,sd


### Completing groupby puzzle #4

Next we'll write code that properly returns _the best result by each party_. That is, each row should show the Year, Candidate, Popular Vote, Result, and % for the election in which that party saw its best results (rather than mixing them as in the example above), here's what the first rows of the correct output should look like:

![parties.png](attachment:ab21f8de-ad29-46c2-bea7-e9aea9c40e3e.png)

In [63]:
elections_sorted_by_percent = elections.sort_values("%", ascending=False)
elections_sorted_by_percent.head(5)

,Year,Candidate,Party,Popular vote,Result,%
114,1964,Lyndon Johnson,Democratic,43127041,win,61.344703
91,1936,Franklin Roosevelt,Democratic,27752648,win,60.978107
120,1972,Richard Nixon,Republican,47168710,win,60.907806
79,1920,Warren Harding,Republican,16144093,win,60.574501
133,1984,Ronald Reagan,Republican,54455472,win,59.023326


In [65]:
elections_sorted_by_percent = elections.sort_values("%", ascending=False)

#takes the first entry (highest based on sort) of each group (in this case by Party)
elections_sorted_by_percent.groupby("Party").agg(lambda x : x.iloc[0])#.head(9)   

In [ ]:
elections_sorted_by_percent = elections.sort_values("%", ascending=False)
elections_sorted_by_percent.groupby("Party").agg(lambda x : x.iloc[0]).head(9)

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1856,Millard Fillmore,873053,loss,21.554001
American Independent,1968,George Wallace,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182
Communist,1932,William Z. Foster,103307,loss,0.261069
Constitution,2008,Chuck Baldwin,199750,loss,0.152398
Constitutional Union,1860,John Bell,590901,loss,12.639283
Democratic,1964,Lyndon Johnson,43127041,win,61.344703


You'll soon discover that with Pandas' rich tool set, there's typically more than one way to get to the same answer. Each approach has different tradeoffs in terms of readability, performance, memory consumption, complexity and more.  It will take some experience for you to develop a sense of which approach is better for each problem, but you should in general try to think if you can at least envision a different solution to a given problem, especially if you find your current solution to be particularly convoluted or hard to read.

Here's a couple of other ways of obtaining the same result (in each case we only show the top part with `head()`).  The first approach uses `groupby` but finds the location of the maximum value via the `idxmax()` method (look up its documentation!).  We then index and sort by party to match the requested formatting:

In [ ]:
elections.head()

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789


In [ ]:
elections.groupby('Party')['%'].idxmax()

Party
American                  22
American Independent     115
Anti-Masonic               6
Anti-Monopoly             38
Citizens                 127
Communist                 89
Constitution             164
Constitutional Union      24
Democratic               114
Democratic-Republican      0
Dixiecrat                103
Farmer–Labor              78
Free Soil                 15
Green                    155
Greenback                 35
Independent              143
Liberal Republican        31
Libertarian              175
National Democratic       50
National Republican        3
National Union            27
Natural Law              148
New Alliance             136
Northern Democratic       26
Populist                  48
Progressive               68
Prohibition               49
Reform                   150
Republican               120
Socialist                 66
Southern Democratic       25
States' Rights           110
Taxpayers                147
Union                     93
Union La

In [ ]:
best_per_party = elections.loc[elections.groupby('Party')['%'].idxmax()]  # This is the computational part
best_per_party.set_index('Party').sort_index().head()  # This indexes by Party to match the formatting above

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1856,Millard Fillmore,873053,loss,21.554001
American Independent,1968,George Wallace,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182


And this one doesn't even use `groupby`! This approach instead uses the `drop_duplicates` method to keep only the last occurrence of of each party after having sorted by "%", which is the best performance.  Again, the 2nd line is purely formatting:

In [ ]:
best_per_party2 = elections.sort_values('%').drop_duplicates(['Party'], keep='last')
best_per_party2.set_index('Party').sort_index().head()  # Formatting

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1856,Millard Fillmore,873053,loss,21.554001
American Independent,1968,George Wallace,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182


*Challenge:* see if you can find yet another approach that still gives the same answer.

---

## Other DataFrameGroupBy Features

The result of groupby is not a DataFrame or a list of DataFrames. It is instead a special type called a `DataFrameGroupBy`.

In [66]:
grouped_by_year = babynames.groupby("Year")
type(grouped_by_year)

pandas.core.groupby.generic.DataFrameGroupBy

### groupby.size()

In [ ]:
elections.groupby("Party")

In [ ]:
#size returns a Series giving the size of each group
elections.groupby("Party").size().head(15)

Party
American                  2
American Independent      3
Anti-Masonic              1
Anti-Monopoly             1
Citizens                  1
Communist                 1
Constitution              3
Constitutional Union      1
Democratic               47
Democratic-Republican     2
Dixiecrat                 1
Farmer–Labor              1
Free Soil                 2
Green                     7
Greenback                 1
dtype: int64

### groupby.filter()

In [67]:
# filter gives a copy of the original DataFrame where row r is included
# if ITS GROUP obeys the given condition
#
# Note: Filtering is done per GROUP, not per ROW.
elections.groupby("Year").filter(lambda sf: sf["%"].max() < 45)  #keeps all rows when the Year had a max party vote < 45%

,Year,Candidate,Party,Popular vote,Result,%
23,1860,Abraham Lincoln,Republican,1855993,win,39.699408
24,1860,John Bell,Constitutional Union,590901,loss,12.639283
25,1860,John C. Breckinridge,Southern Democratic,848019,loss,18.138998
26,1860,Stephen A. Douglas,Northern Democratic,1380202,loss,29.522311
66,1912,Eugene V. Debs,Socialist,901551,loss,6.004354
67,1912,Eugene W. Chafin,Prohibition,208156,loss,1.386325
68,1912,Theodore Roosevelt,Progressive,4122721,loss,27.457433
69,1912,William Taft,Republican,3486242,loss,23.218466
70,1912,Woodrow Wilson,Democratic,6296284,win,41.933422
115,1968,George Wallace,American Independent,9901118,loss,13.571218


In [69]:
# the code below lets us peek into the groups and see why they were rejected or not
for i, (n, g) in enumerate(elections.groupby("Year")):
    print(n)
    display(g)
    if i>3: break

1824


,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878


1828


,Year,Candidate,Party,Popular vote,Result,%
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073


1832


,Year,Candidate,Party,Popular vote,Result,%
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
5,1832,Henry Clay,National Republican,484205,loss,37.603628
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583


1836


,Year,Candidate,Party,Popular vote,Result,%
7,1836,Hugh Lawson White,Whig,146109,loss,10.005985
8,1836,Martin Van Buren,Democratic,763291,win,52.272472
9,1836,William Henry Harrison,Whig,550816,loss,37.721543


1840


,Year,Candidate,Party,Popular vote,Result,%
10,1840,Martin Van Buren,Democratic,1128854,loss,46.948787
11,1840,William Henry Harrison,Whig,1275583,win,53.051213


## Puzzle 5: Finding the number of babies born in each year of each sex.

Earlier we saw how to add up the total number of babies born in each year, but what if we want the number born in each and year for each sex separately?

In [ ]:
babynames.groupby("Year").sum()

,Count
Year,
1910,9163
1911,9983
1912,17946
1913,22094
1914,26926
...,...
2016,426658
2017,410614
2018,394826


It is possible to group a DataFrame by multiple features. For example, if we group by Year and Sex we get back a DataFrame with the total number of babies of each sex born in each year.

In [ ]:
babynames.head(10)

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
5,CA,F,1910,Ruth,128
6,CA,F,1910,Evelyn,126
7,CA,F,1910,Alice,118
8,CA,F,1910,Virginia,101
9,CA,F,1910,Elizabeth,93


In [ ]:
babynames.groupby(["Year", "Sex"]).agg(sum).head(6)

Count
Year Sex       
1910 F     5950
     M     3213
1911 F     6602
     M     3381
1912 F     9804
     M     8142

In [ ]:
babynames.groupby(["Sex", "Year"]).agg(sum).head(6)

Count
Sex Year       
F   1910   5950
    1911   6602
    1912   9804
    1913  11860
    1914  13815
    1915  18643

The DataFrame resulting from an aggregation operation on a table grouped by a list of columns is multi-indexed. That is, it has more than one dimension to its index. We will explore this in a future lecture.

A more natural approach is to use a pivot table (like we saw in data 8).

In [ ]:
babynames.head(5)

,State,Sex,Year,Name,Count
131013,CA,F,1994,Leandrea,5
115942,CA,F,1990,Deandrea,5
300700,CA,M,1985,Deandrea,6
108715,CA,F,1988,Deandrea,5
101962,CA,F,1986,Deandrea,6


In [ ]:
babynames_pivot = babynames.pivot_table(
    index='Year',     # the rows (turned into index)
    columns='Sex',    # the column values
    values=['Count'], # the field(s) to processed in each group
    aggfunc=np.sum,   # group operation
)
babynames_pivot.head(6)

Sex,F,M
Year,,
1910,5950,3213
1911,6602,3381
1912,9804,8142
1913,11860,10234
1914,13815,13111
1915,18643,17192


In [ ]:
babynames_pivot = babynames.pivot_table(
    index='Year',     # the rows (turned into index)
    columns='Sex',    # the column values
    values=['Count', 'Name'], 
    aggfunc=np.max,   # group operation
)
babynames_pivot.head(6)

Count          Name         
Sex      F     M       F        M
Year                             
1910   295   237  Yvonne  William
1911   390   214   Zelma   Willis
1912   534   501  Yvonne  Woodrow
1913   584   614   Zelma   Yoshio
1914   773   769   Zelma   Yoshio
1915   998  1033    Zita    Yukio

The basic idea behind pivot tables is shown in the image below.

![pivot_picture.png](attachment:pivot_picture.png)

---

## Merging Tables

In [ ]:
elections

NameError: name 'elections' is not defined

In [ ]:
male_2020_babynames = babynames.query('Sex == "M" and Year == 2020')
male_2020_babynames

,State,Sex,Year,Name,Count
392447,CA,M,2020,Deandre,19
394024,CA,M,2020,Leandre,5
392438,CA,M,2020,Andreas,19
391863,CA,M,2020,Leandro,72
392562,CA,M,2020,Rudra,17
...,...,...,...,...,...
393418,CA,M,2020,Demetri,7
393881,CA,M,2020,Casimir,5
393393,CA,M,2020,Bridger,7
393391,CA,M,2020,Brayson,7


In [ ]:
elections["First Name"] = elections["Candidate"].str.split().str[0]

In [ ]:
elections

,Year,Candidate,Party,Popular vote,Result,%,First Name
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,Andrew
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,John
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,Andrew
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,John
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,Andrew
...,...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699,Jill
178,2020,Joseph Biden,Democratic,81268924,win,51.311515,Joseph
179,2020,Donald Trump,Republican,74216154,loss,46.858542,Donald
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979,Jo


In [ ]:
merged = pd.merge(left = elections, right = male_2020_babynames, 
                  left_on = "First Name", right_on = "Name")
merged

NameError: name 'male_2020_babynames' is not defined

In [ ]:
merged.sort_values("Count")

,Year_x,Candidate,Party,Popular vote,Result,%,First Name,State,Sex,Year_y,Name,Count
129,1996,Bill Clinton,Democratic,47400125,win,49.296938,Bill,CA,M,2020,Bill,5
106,1956,Dwight Eisenhower,Republican,35579180,win,57.650654,Dwight,CA,M,2020,Dwight,5
105,1952,Dwight Eisenhower,Republican,34075529,win,55.325173,Dwight,CA,M,2020,Dwight,5
131,1992,Ross Perot,Independent,19743821,loss,18.956298,Ross,CA,M,2020,Ross,5
132,1996,Ross Perot,Reform,8085294,loss,8.408844,Ross,CA,M,2020,Ross,5
...,...,...,...,...,...,...,...,...,...,...,...,...
121,2004,David Cobb,Green,119859,loss,0.098088,David,CA,M,2020,David,1155
120,1984,David Bergland,Libertarian,228111,loss,0.247245,David,CA,M,2020,David,1155
66,1884,Benjamin Butler,Anti-Monopoly,134294,loss,1.335838,Benjamin,CA,M,2020,Benjamin,1634
67,1888,Benjamin Harrison,Republican,5443633,win,47.858041,Benjamin,CA,M,2020,Benjamin,1634
